In [ ]:
execfile('d:/measuring/analysis/scripts/setup_analysis.py')
%matplotlib inline


In [ ]:
nr_of_scans = 16
older_than=None
fig,ax=plt.subplots(1, figsize=(16,5))
for i in range(nr_of_scans):
    timestamp,folder = tb.latest_data(contains='LaserScan', older_than = older_than,return_timestamp=True)
    older_than=timestamp

    #print folder

    a=m2.M2Analysis(folder=folder)
    for k in a.f.keys():
        if '_data' in k:
            a.dg = a.f[k]
    a.counts,a.frequency,a.voltage = a.dg['Counts [Hz]'].value,a.dg['Frequency [GHz]'].value,a.dg['Voltage [V]'].value
    #print len(a.counts)
    if len(a.counts)>200:
        ax.plot(a.frequency, a.counts, label=' '+a.name)
        
ax.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax.set_ylabel('Countrate [Hz]')
#ax.legend()

In [ ]:
fig,ax=plt.subplots(1)


In [ ]:
tb.get_msmt_fp?

In [ ]:
nr_of_scans = 16
older_than=None
newer_than = '20160913132000'
newer_than_old = '20160908080000'
fig,ax=plt.subplots(1, figsize=(16,5))
fig2,ax2=plt.subplots(1, figsize=(16,5))
bins = np.arange(-500,500,1.)
hist = np.zeros(len(bins)-1)
bin_norm = np.zeros(len(bins)-1)
while 1:
    old_laser_scan=False
    ret = tb.latest_data(contains='LaserScan', older_than = older_than,newer_than=newer_than,return_timestamp=True, raise_exc=False)
    if not ret:
        ret = tb.latest_data(contains='ldfsaaser_scan', older_than = older_than,newer_than=newer_than_old,return_timestamp=True, raise_exc=False)
        old_laser_scan = True
        if not ret:
            break
    timestamp,folder = ret
    older_than=timestamp
    if old_laser_scan:
        d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
        if len(d)>2:
            voltage, frequency, counts = d[:,0],d[:,1],d[:,2]
    else:
        a=m2.M2Analysis(folder=folder)
        data_found = False
        for k in a.f.keys():
            if '_data' in k and 'area_5' in k:
                a.dg = a.f[k]
                data_found=True
        if not data_found:
            continue
        voltage, frequency, counts = a.dg['Voltage [V]'].value, a.dg['Frequency [GHz]'].value, a.dg['Counts [Hz]'].value
     
    if len(counts)>200:
        cts_norm_zero = counts.astype(np.float)/np.average(counts) -1
        _h,_b = np.histogram(frequency, weights=cts_norm_zero, bins=bins)
        bin_norm+=1.0*(_h>0)
        hist+= _h
        ax2.plot(frequency,cts_norm_zero+1, alpha=0.5)
        
ax.plot(bins[1:],hist, '.')        
ax.set_xlim(-500,500)
ax.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax.set_ylabel('Deviation from average countrates')
ax2.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax2.set_ylabel('Normalized countrates')
ax2.set_xlim(-500,500)
#ax.legend()

In [ ]:
bin_norm[100:]